In [1]:
pip install kagglehub


Note: you may need to restart the kernel to use updated packages.


In [2]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from prophet import Prophet
import datetime as dt
import kagglehub as k
import json
from dotenv import load_dotenv
import os
from pathlib import Path




In [6]:
movies_box_office_2000_2009_path = Path('2000-2009 Movies Box Ofice Collection.csv')
movies_box_office_2010_2024_path = Path('2010-2024 Movies Box Ofice Collection.csv')

mojo_budget_data_path = Path('Mojo_budget_data.csv')
mojo_budget_update_path = Path('Mojo_budget_update.csv')

movies_box_office_2000_2009_data = pd.read_csv(movies_box_office_2000_2009_path)
movies_box_office_2010_2024_data = pd.read_csv(movies_box_office_2010_2024_path)
# mojo_budget_data = pd.read_csv(mojo_budget_data_path)
mojo_budget_update_data = pd.read_csv(mojo_budget_update_path)

# if mojo_budget_data_path.exists():
# 	mojo_budget_data = pd.read_csv(mojo_budget_data_path)
# else:
# 	print(f"File not found: {mojo_budget_data_path}")

# if mojo_budget_update_path.exists():
# 	mojo_budget_update = pd.read_csv(mojo_budget_update_path)
# else:
# 	print(f"File not found: {mojo_budget_update_path}")


In [ ]:
# kaggle_json_path = Path('kaggle.json')
# load_dotenv()
# kaggle_json_path = os.getenv('kaggle_json_path')

In [10]:
mojo_budget_update_data.columns

Index(['movie_id', 'title', 'year', 'trivia', 'mpaa', 'release_date',
       'run_time', 'distributor', 'director', 'writer', 'producer', 'composer',
       'cinematographer', 'main_actor_1', 'main_actor_2', 'main_actor_3',
       'main_actor_4', 'budget', 'domestic', 'international', 'worldwide',
       'genre_1', 'genre_2', 'genre_3', 'genre_4', 'html'],
      dtype='object')

In [85]:
movies_box_office_2000_2009_data = movies_box_office_2000_2009_data.dropna()
movies_box_office_2000_2009_data

,Unnamed: 0,Release Group,Worldwide,Domestic,Domestic_percent,Foreign,Foreign_percent,year
0,0,Mission: Impossible II,"54,63,88,108","21,54,09,889",39.40%,"33,09,78,219",60.60%,2000
1,1,Gladiator,"46,05,83,960","18,77,05,427",40.80%,"27,28,78,533",59.20%,2000
2,2,Cast Away,"42,96,32,142","23,36,32,142",54.40%,"19,60,00,000",45.60%,2000
3,3,What Women Want,"37,41,11,707","18,28,11,707",48.90%,"19,13,00,000",51.10%,2000
4,4,Dinosaur,"34,98,22,765","13,77,48,063",39.40%,"21,20,74,702",60.60%,2000
...,...,...,...,...,...,...,...,...
1995,195,Kamen Rider Decade: All Riders vs. Dai0Shocker,"1,94,18,091",0,0,"1,94,18,091",100%,2009
1996,196,Ballad,"1,93,28,678",0,0,"1,93,28,678",100%,2009
1997,197,The White Ribbon,"1,93,19,671","22,22,862",11.50%,"1,70,96,809",88.50%,2009
1998,198,Cell 211,"1,93,00,483",0,0,"1,93,00,483",100%,2009


In [89]:
movies_box_office_2010_2024_data = movies_box_office_2010_2024_data.dropna()
movies_box_office_2010_2024_data.head()

,Rank,Release Group,Worldwide,Domestic,Domestic_percent,Foreign,Foreign_percent,year
0,0,Toy Story 3,"1,06,69,69,703","41,50,04,880",38.90%,"65,19,64,823",61.10%,2010
1,1,Alice in Wonderland,"1,02,54,67,110","33,41,91,110",32.60%,"69,12,76,000",67.40%,2010
2,2,Harry Potter and the Deathly Hallows: Part 1,"96,02,83,305","29,59,83,305",30.80%,"66,43,00,000",69.20%,2010
3,3,Inception,"82,82,58,695","29,25,76,195",35.30%,"53,56,82,500",64.70%,2010
4,4,Shrek Forever After,"75,26,00,867","23,87,36,787",31.70%,"51,38,64,080",68.30%,2010


In [102]:
joined_movies_box_office_data = pd.concat([movies_box_office_2000_2009_data, movies_box_office_2010_2024_data], 
                                          axis ="rows", join ="inner").rename(columns={'year': 'Year', 'Release Group' : 'Title'})
                                            
joined_movies_box_office_data.head()

,Title,Worldwide,Domestic,Domestic_percent,Foreign,Foreign_percent,Year
0,Mission: Impossible II,"54,63,88,108","21,54,09,889",39.40%,"33,09,78,219",60.60%,2000
1,Gladiator,"46,05,83,960","18,77,05,427",40.80%,"27,28,78,533",59.20%,2000
2,Cast Away,"42,96,32,142","23,36,32,142",54.40%,"19,60,00,000",45.60%,2000
3,What Women Want,"37,41,11,707","18,28,11,707",48.90%,"19,13,00,000",51.10%,2000
4,Dinosaur,"34,98,22,765","13,77,48,063",39.40%,"21,20,74,702",60.60%,2000


In [92]:
joined_movies_box_office_data.info()




<class 'pandas.core.frame.DataFrame'>
Index: 4800 entries, 0 to 2799
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Release Group     4800 non-null   object
 1   Worldwide         4800 non-null   object
 2   Domestic          4800 non-null   object
 3   Domestic_percent  4800 non-null   object
 4   Foreign           4800 non-null   object
 5   Foreign_percent   4800 non-null   object
 6   year              4800 non-null   int64 
dtypes: int64(1), object(6)
memory usage: 300.0+ KB


KeyError: 'Release Date'